In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tnrange
import matplotlib.pyplot as plt

import gym.envs.classic_control.cartpole

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Environments

In [2]:
""" Four Rooms Environment Implementation
"""
class FourRooms(object):
    def __init__(self, max_time_steps=459):
        # We define the grid for the Four Rooms domain
        self.grid = np.array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

        # We define the observation space consisting of all empty cells
        # Note: We have to flip the coordinates from (row_idx, column_idx) -> (x, y),
        # where x = column_idx, y = 10 - row_idx
        self.observation_space = np.argwhere(self.grid == 0.0).tolist()  # Fine all empty cells
        self.observation_space = self.arr_coords_to_four_room_coords(self.observation_space)

        # We define the action space
        self.action_space = {'up': np.array([0, 1]),
                             'down': np.array([0, -1]),
                             'left': np.array([-1, 0]),
                             'right': np.array([1, 0])}
        self.action_names = ['up', 'down', 'left', 'right']

        # We define the start location
        self.start_location = [0, 0]

        # We define the goal location
        self.goal_location = [10, 10]

        # We find all wall cells
        self.walls = np.argwhere(self.grid == 1.0).tolist()  # find all wall cells
        self.walls = self.arr_coords_to_four_room_coords(self.walls)  # convert to Four Rooms coordinates

        # This is an episodic task, we define a timeout: maximal time steps = 459
        self.max_time_steps = max_time_steps

        # We define other useful variables
        self.agent_location = None  # track the agent's location in one episode.
        self.action = None  # track the agent's action
        self.t = 0  # track the current time step in one episode

    @staticmethod
    def arr_coords_to_four_room_coords(arr_coords_list):
        """
        Function converts the array coordinates to the Four Rooms coordinates (i.e, The origin locates at bottom left).
        E.g., The coordinates (0, 0) in the numpy array is mapped to (0, 10) in the Four Rooms coordinates.
        Args:
            arr_coords_list (list): a list variable consists of tuples of locations in the numpy array

        Return:
            four_room_coords_list (list): a list variable consists of tuples of converted locations in the
                                          Four Rooms environment.
        """
        # Note: We have to flip the coordinates from (row_idx, column_idx) -> (x, y),
        # where x = column_idx, y = 10 - row_idx
        four_room_coords_list = [(column_idx, 10 - row_idx) for (row_idx, column_idx) in arr_coords_list]
        return four_room_coords_list

    def reset(self):
        # We reset the agent's location to the start location
        self.agent_location = self.start_location

        # We reset the timeout tracker to be 0
        self.t = 0

        # We set the information
        info = {}
        return self.agent_location, info

    def step(self, action):
        """
        Args:
            action (string): a string variable (i.e., "UP"). All feasible values are ["up", "down", "left", "right"].
        """
        # With probability 0.8, the agent takes the correct direction.
        # With probability 0.2, the agent takes one of the two perpendicular actions.
        # For example, if the correct action is "LEFT", then
        #     - With probability 0.8, the agent takes action "LEFT";
        #     - With probability 0.1, the agent takes action "UP";
        #     - With probability 0.1, the agent takes action "DOWN".
        if np.random.uniform() < 0.2:
            if action == "left" or action == "right":
                action = np.random.choice(["up", "down"], 1)[0]
            else:
                action = np.random.choice(["right", "left"], 1)[0]

        # Convert the agent's location to array
        loc_arr = np.array(self.agent_location)

        # Convert the action name to movement array
        act_arr = self.action_space[action]

        # Compute the agent's next location
        next_agent_location = np.clip(loc_arr + act_arr,
                                      a_min=np.array([0, 0]),
                                      a_max=np.array([10, 10])).tolist()

        # Check if the agent crashes into walls, it stays at the current location.
        if tuple(next_agent_location) in self.walls:
            next_agent_location = self.agent_location

        # Compute the reward
        reward = 1.0 if next_agent_location == self.goal_location else 0.0

        # Check the termination
        # If the agent reaches the goal, reward = 1, done = True
        # If the time steps reaches the maximal number, reward = 0, done = True.
        if reward == 1.0 or self.t == self.max_time_steps:
            terminated = True
        else:
            terminated = False

        # Update the agent's location, action and time step trackers
        self.agent_location = next_agent_location
        self.action = action
        self.t += 1

        return next_agent_location, reward, terminated, False, {}

    def render(self):
        # plot the agent and the goal
        # empty cell = 0
        # wall cell = 1
        # agent cell = 2
        # goal cell = 3
        plot_arr = self.grid.copy()
        plot_arr[10 - self.agent_location[1], self.agent_location[0]] = 2
        plot_arr[10 - self.goal_location[1], self.goal_location[0]] = 3
        plt.clf()
        plt.title(f"state={self.agent_location}, act={self.action}")
        plt.imshow(plot_arr)
        plt.show(block=False)
        plt.pause(0.1)

    @staticmethod
    def test():
        my_env = FourRooms()
        state, _ = my_env.reset()

        for _ in range(100):
            action = np.random.choice(list(my_env.action_space.keys()), 1)[0]

            next_state, reward, done, _, _ = my_env.step(action)
            my_env.render()

            if done:
                state, _ = my_env.reset()
            else:
                state = next_state

In [3]:
class StochasticWindyGridWorld(object):
    def __init__(self):
        # We define the grid for the DynaMaze domain
        self.grid = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
        self.grid_height, self.grid_width = self.grid.shape

        # We define the observation space consisting of all empty cells
        self.observation_space = np.argwhere(self.grid == 0.0).tolist()

        # We define the action space
        self.action_space = {
            "up": np.array([-1, 0]),
            "down": np.array([1, 0]),
            "left": np.array([0, -1]),
            "right": np.array([0, 1])
        }
        # We define the action names
        self.action_names = ['up', 'down', 'left', 'right']

        # We define the start state
        self.start_location = [3, 0]

        # We define the goal state
        self.goal_location = [3, 7]

        # Define the wind list
        self.wind_space = (0, 0, 0, 1, 1, 1, 2, 2, 1, 0)

        # We define other useful variables
        self.agent_location = None
        self.action = None

    def reset(self):
        # We reset the agent location to the start state
        self.agent_location = self.start_location

        # We set the information
        info = {}
        return self.agent_location, info

    def step(self, action):
        """
        Args:
            action (string): name of the action (e.g., "up"/"down"/"left"/"right")
        """
        # Convert the agent's location to array
        loc_arr = np.array(self.agent_location)

        # Convert the action name to movement array
        act_arr = self.action_space[action]

        # Make the wind stochastic
        mean_wind = self.wind_space[self.agent_location[1]]
        if mean_wind:  # if the wind exists
            wind = np.random.choice([mean_wind - 1,
                                     mean_wind,
                                     mean_wind + 1],
                                    p=[1.0/3.0, 1.0/3.0, 1.0/3.0])
        else:
            wind = 0
        # Compute the wind array
        wind_arr = -1 * np.array([wind, 0], dtype=int)

        # compute the next state
        next_agent_location = np.clip(loc_arr + act_arr + wind_arr,
                                      a_min=np.array([0, 0]),
                                      a_max=np.array([self.grid_height - 1, self.grid_width - 1])).tolist()

        # Compute the reward
        reward = 1 if next_agent_location == self.goal_location else 0

        # Check the termination
        terminated = True if reward == 1 else False

        # Update the action location
        self.agent_location = next_agent_location
        self.action = action

        return next_agent_location, reward, terminated, False, {}

    def render(self):
        # plot the agent and the goal
        # agent = 1
        # goal = 2
        plot_arr = self.grid.copy()
        plot_arr[self.agent_location[0], self.agent_location[1]] = 2
        plot_arr[self.goal_location[0], self.goal_location[1]] = 3
        plt.clf()
        plt.title(f"state={self.agent_location}, act={self.action}")
        plt.imshow(plot_arr)
        plt.show(block=False)
        plt.pause(0.5)

In [4]:
class BlockingMaze(object):
    """ Implementation of the Blocking Maze """
    def __init__(self):
        # We define the grid for the maze on the left in Example 8.2
        self.left_grid = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
                                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                                   [1, 1, 1, 1, 1, 1, 1, 1, 0],
                                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                                   [0, 0, 0, 0, 0, 0, 0, 0, 0]])

        # We define the grid for the maze on the right in Example 8.2
        self.right_grid = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 1, 1, 1, 1, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0]])

        # We initialize the grid with using the left maze first
        self.grid = self.left_grid.copy()
        # Save the size of the maze
        self.grid_height, self.grid_width = self.grid.shape

        # We define the observation space using all empty cells.
        # We consider all cells. Because the environment switch will results in change of the state space
        self.observation_space = np.argwhere(np.zeros((self.grid_height, self.grid_width)) == 0.0).tolist()

        # We define the action space
        self.action_space = {
            "up": np.array([-1, 0]),
            "down": np.array([1, 0]),
            "left": np.array([0, -1]),
            "right": np.array([0, 1])
        }
        self.action_names = ["up", "down", "left", "right"]

        # We define the start state
        self.start_location = [5, 3]

        # We define the goal state
        self.goal_location = [0, 8]

        # We find all wall locations in the grid
        self.walls = np.argwhere(self.grid == 1.0).tolist()

        # We define other useful variables
        self.agent_location = None
        self.action = None

    def reset(self, shift_maze=None):
        """
        Args:
            shift_maze (string): if None, reset function simply set the agent back to the start location
                                 if left, reset function will switch to the maze on the left
                                 if right, reset function will switch to the maze on the right
        """
        if shift_maze is not None:
            if shift_maze == "left":
                self.grid = self.left_grid.copy()
            elif shift_maze == "right":
                self.grid = self.right_grid.copy()
            else:
                raise Exception("Invalid shift operation")

            # reset the shape
            self.grid_height, self.grid_width = self.grid.shape

            # reset the walls
            self.walls = np.argwhere(self.grid == 1.0).tolist()

        # We reset the agent location to the start state
        self.agent_location = self.start_location

        # We set the information
        info = {}
        return self.agent_location, info

    def step(self, action):
        """
        Args:
            action (string): name of the action (e.g., "up"/"down"/"left"/"right")
        """
        # Convert the agent's location to array
        loc_arr = np.array(self.agent_location)

        # Convert the abstract action to movement array
        act_arr = self.action_space[action]

        # Compute the next location
        next_agent_location = np.clip(loc_arr + act_arr,
                                      a_min=np.array([0, 0]),
                                      a_max=np.array([self.grid_height - 1, self.grid_width - 1])).tolist()

        # Check if it crashes into a wall
        if next_agent_location in self.walls:
            # If True, the agent keeps in the original state
            next_agent_location = self.agent_location

        # Compute the reward
        reward = 1 if next_agent_location == self.goal_location else 0

        # Check the termination
        terminated = True if reward == 1 else False

        # Update the action location
        self.agent_location = next_agent_location
        self.action = action

        return next_agent_location, reward, terminated, False, {}

    def render(self):
        # plot the agent and the goal
        # agent = 1
        # goal = 2
        plot_arr = self.grid.copy()
        plot_arr[self.agent_location[0], self.agent_location[1]] = 2
        plot_arr[self.goal_location[0], self.goal_location[1]] = 3
        plt.clf()
        plt.title(f"state={self.agent_location}, act={self.action}")
        plt.imshow(plot_arr)
        plt.show(block=False)
        plt.pause(0.01)

# PPO

In [5]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0.0, std=0.1)
        nn.init.constant_(m.bias, 0.1)

# Define actor-critic network, subclass of torch module
class ActorCriticNetwork(nn.Module):

    # Subclass of torch module
    def __init__(self, num_inputs, num_outputs, num_hidden):
        super(ActorCriticNetwork, self).__init__()

        # Define actor network
        self.actor = nn.Sequential(nn.Linear(num_inputs, num_hidden), nn.ReLU(), nn.Linear(num_hidden, num_outputs))

        # Define critic network
        self.critic = nn.Sequential(nn.Linear(num_inputs, num_hidden), nn.ReLU(), nn.Linear(num_hidden, 1))

        # Initialize weights
        self.apply(initialize_weights)

    # Define forward propagation
    def forward(self, state):
        action_vals = self.actor(state)
        action_probs = F.softmax(action_vals, dim=0)
        estimated_value = self.critic(state)
        return action_probs, estimated_value

In [6]:
def compute_returns(rewards, gamma=0.99):
    G = 0
    returns = np.zeros(len(rewards))
    for i in reversed(range(len(rewards))):
        G = gamma * G + rewards[i]
        returns[i] = G

    np.reshape(returns, (-1,1))
    return returns

In [7]:
# Define PPO loss function (takes episodic data)
def ppo_loss(model, optimizer, states, actions, old_action_probs, returns, clip_param=0.2, c1=0.5, c2=0.001):

    # Convert lists to tensors
    # states = torch.FloatTensor(states)
    states = torch.FloatTensor(np.array(states))
    actions = torch.unsqueeze(torch.tensor(actions, dtype=int), dim=1) # actions must be int-type tensor for indexing
    old_action_probs = torch.vstack(old_action_probs) # actions probs passed in as list of tensors
    returns = torch.FloatTensor(returns) # returns are list of floats

    # Obtain value estimates and action predictions for these states
    new_action_probs, values = model(states)

    # Select correct action probability from set of action probabilities by indexing using actions
    new_action_probs = torch.gather(new_action_probs, dim=1, index=actions)

    # Compute probability ratio
    r = new_action_probs / old_action_probs

    # Compute advantages
    advantages = returns - values

    # Compute actor (clipped) loss
    surr1 = r * advantages
    surr2 = torch.clamp(r, 1 - clip_param, 1 + clip_param) * advantages
    clipped_loss =  (torch.min(surr1, surr2)).mean()

    # Compute critic loss
    critic_loss = advantages.pow(2).mean()

    # Compute entropy term
    entropy = torch.zeros(1)

    # Compute loss
    loss = clipped_loss + c1*critic_loss - c2*entropy
    
    # Compute gradient of loss function and take step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss

In [28]:
def train_model_episodic(env, num_episodes, max_episode_length, learning_rate=3E-4):
    
    # Initialize results arrays
    mean_rewards, total_rewards, losses = np.zeros(num_episodes), np.zeros(num_episodes), np.zeros(num_episodes)
    
    # Initialize new model and optimizer
    model = ActorCriticNetwork(num_inputs, num_outputs, num_hidden)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, maximize=True)
    
    # Iterate for number of episodes
    states, actions, rewards, old_action_probs = [], [], [], []
    for ep in tnrange(num_episodes):
    
        # Generate episode
        state, _ = env.reset()
        done = False
        steps = 0
        while (not done) and (steps < max_episode_length):
    
            # get action probabilities and estimate state value from model
            if num_inputs == 1:
                state = [state]
            action_probs, _ = model(torch.FloatTensor(state))
    
            # select action using action probabilities
            action = np.random.choice(list(range(num_outputs)), p=action_probs.detach().numpy())
            
            # take a step and observe next state, reward
            if isinstance(env, BlockingMaze) or isinstance(env, FourRooms) or isinstance(env, StochasticWindyGridWorld):
                next_state, reward, done, _, _ = env.step(list(env.action_space.keys())[action])
            else:
                next_state, reward, done, _, _ = env.step(action)
    
            # append state, reward, and action probabilities
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            old_action_probs.append(action_probs[action])
    
            # update state
            state = next_state
            
            # increment steps
            steps += 1
        
        # Update results
        mean_rewards[ep] += np.mean(rewards)
        total_rewards[ep] += np.sum(rewards)
    
        # compute returns
        returns = compute_returns(rewards)
    
        # call to PPO update to update weights
        for _ in range(epochs):
            loss = ppo_loss(model, optimizer, states, actions, old_action_probs, returns)
            losses[ep] += loss.detach().numpy()
    
        # reset state, actions, reward, action prob tracking arrays
        states, actions, rewards, old_action_probs = [], [], [], []

    return mean_rewards, total_rewards, losses

In [15]:
def train_model_stepwise(env, num_steps, max_episode_length, learning_rate=3E-4):

    # Initialize results array
    cumulative_rewards = np.zeros(num_steps + 1)

    # Initialize new model and optimizer
    model = ActorCriticNetwork(num_inputs, num_outputs, num_hidden)
    # optimizer = optim.Adam(model.parameters(), lr=learning_rate, maximize=True)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # reset state, actions, reward, action prob tracking arrays
    states, actions, rewards, old_action_probs = [], [], [], []

    # reset environment
    state, _ = env.reset()
    done = False

    # reset episode steps
    episode_step = 0

    # Iterate for number of steps
    for step in tnrange(num_steps):

        # get action probabilities and estimate state value from model
        if num_inputs == 1:
            state = [state]
        action_probs, _ = model(torch.FloatTensor(state))

        # select action using action probabilities
        action = np.random.choice(list(range(num_outputs)), p=action_probs.detach().numpy())

        # take a step and observe next state, reward
        if isinstance(env, BlockingMaze) or isinstance(env, FourRooms) or isinstance(env, StochasticWindyGridWorld):
            next_state, reward, done, _, _ = env.step(list(env.action_space.keys())[action])
        else:
            next_state, reward, done, _, _ = env.step(action)

        # append state, reward, and action probabilities
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        old_action_probs.append(action_probs[action])

        # update state
        state = next_state

        # increment steps
        episode_step += 1

        # add reward to cumulative rewards
        cumulative_rewards[step + 1] = cumulative_rewards[step] + reward

        # print('State: ' + str(state) + '\tAction: ' + str(action) + '\tProbs: ' + str(action_probs.detach().numpy()))
        # if reward == 1:
        #     print(step)

        # Check if episode complete
        if done or (episode_step > max_episode_length):

            # compute returns
            returns = compute_returns(rewards)

            # call to PPO update to update weights
            for _ in range(epochs):
                ppo_loss(model, optimizer, states, actions, old_action_probs, returns)

            # reset state, actions, reward, action prob tracking arrays
            states, actions, rewards, old_action_probs = [], [], [], []

            # reset environment
            state, _ = env.reset()
            done = False

            # reset episode steps
            episode_step = 0

    return cumulative_rewards[1:]

In [29]:
# Initialize environment

env = gym.make('CartPole-v1')
num_inputs  = 4
num_outputs = 2
num_hidden  = 256

# env = FourRooms(max_time_steps=sys.maxsize)
# num_inputs  = 2
# num_outputs = 4
# num_hidden  = 256

# env = BlockingMaze()
# num_inputs  = 2
# num_outputs = 4
# num_hidden  = 256

# env = StochasticWindyGridWorld()
# num_inputs  = 2
# num_outputs = 4
# num_hidden  = 256

In [32]:
# Main training loop (stepwise)
num_steps = 300000
num_runs = 5
epochs = 1
# max_episode_length = sys.maxsize
max_episode_length = 10000

cumulative_rewards = []
for _ in tnrange(num_runs):
    rewards = train_model_stepwise(env, num_steps, max_episode_length, learning_rate=3E-4)
    cumulative_rewards.append(rewards)

In [43]:
# Main training loop (episodic)
num_episodes = 3000
num_runs = 5
epochs = 1
# max_episode_length = sys.maxsize
max_episode_length = 1000

means, totals, losses = [], [], []
for _ in tnrange(num_runs):
    mean_rewards, total_rewards, l = train_model_episodic(env, num_episodes, max_episode_length, learning_rate=3E-4)
    means.append(mean_rewards)
    totals.append(total_rewards)
    losses.append(l)

In [ ]:
ppo_cartpole = pd.read_csv('/content/ppo_cartpole_final.csv')
a2c_cartpole = pd.read_csv('/content/a2c_cartpole_final.csv', index_col=0)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,4), dpi=150)
plt.plot(np.mean(ppo_cartpole, axis=1))
plt.plot(np.mean(a2c_cartpole, axis=1), alpha=0.9)
plt.title('Total reward per episode, CartPole-v1 (average of 5 runs)')
plt.xlabel('Episode')
plt.ylabel('Total reward per episode')
plt.legend(['PPO','A2C'])
plt.show()

In [ ]:
ppo_windy = pd.read_csv('/content/ppo_windy_final.csv')
a2c_windy = pd.read_csv('/content/a2c_windy_final.csv', index_col=0)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,4), dpi=150)
plt.plot(np.mean(ppo_windy, axis=1)[:150000])
plt.plot(np.mean(a2c_windy, axis=1))
plt.title('Cumulative reward, Stochastic Windy Gridworld (average of 5 runs)')
plt.xlabel('Step')
plt.ylabel('Cumulative Reward')
plt.legend(['PPO','A2C'])
plt.show()

In [ ]:
ppo_4rooms = pd.read_csv('/content/ppo_4rooms_final.csv')
a2c_4rooms = pd.read_csv('/content/a2c_4rooms_final.csv', index_col=0)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,4), dpi=150)
plt.plot(np.mean(ppo_4rooms, axis=1)[:150000])
plt.plot(np.mean(a2c_4rooms, axis=1))
plt.title('Cumulative reward, Four Rooms (average of 5 runs)')
plt.xlabel('Step')
plt.ylabel('Cumulative Reward')
plt.legend(['PPO','A2C'])
plt.show()

In [ ]:
ppo_blocking = pd.read_csv('/content/ppo_blocking_final.csv')
a2c_blocking = pd.read_csv('/content/a2c_blocking_final.csv', index_col=0)


fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,4), dpi=150)
plt.plot(np.mean(ppo_blocking, axis=1)[:150000])
plt.plot(np.mean(a2c_blocking, axis=1))
plt.title('Cumulative reward, Blocking Maze (average of 5 runs)')
plt.xlabel('Step')
plt.ylabel('Cumulative Reward')
plt.legend(['PPO','A2C'])
plt.show()